In [13]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.metrics import roc_auc_score, classification_report
import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv("../data/processed_data.csv")

X = df.drop("Churn", axis=1)
y = df["Churn"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [3]:
logreg = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)

param_grid_lr = {
    "C": [0.01, 0.1, 1, 10],
    "penalty": ["l1", "l2"],
    "solver": ["liblinear"]
}

grid_lr = GridSearchCV(logreg, param_grid_lr, cv=5, scoring='roc_auc', n_jobs=-1)
grid_lr.fit(X_train, y_train)

print("📌 Best Params - Logistic Regression")
print(grid_lr.best_params_)

y_pred_lr = grid_lr.predict(X_test)
y_proba_lr = grid_lr.predict_proba(X_test)[:, 1]
print(classification_report(y_test, y_pred_lr))
print(f"ROC AUC: {roc_auc_score(y_test, y_proba_lr):.4f}")


📌 Best Params - Logistic Regression
{'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
              precision    recall  f1-score   support

           0       0.90      0.72      0.80      1035
           1       0.50      0.78      0.61       374

    accuracy                           0.74      1409
   macro avg       0.70      0.75      0.71      1409
weighted avg       0.80      0.74      0.75      1409

ROC AUC: 0.8406


In [4]:
rf = RandomForestClassifier(class_weight="balanced", random_state=42)

param_grid_rf = {
    "n_estimators": [100, 200],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5],
}

grid_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='roc_auc', n_jobs=-1)
grid_rf.fit(X_train, y_train)

print("📌 Best Params - Random Forest")
print(grid_rf.best_params_)

y_pred_rf = grid_rf.predict(X_test)
y_proba_rf = grid_rf.predict_proba(X_test)[:, 1]
print(classification_report(y_test, y_pred_rf))
print(f"ROC AUC: {roc_auc_score(y_test, y_proba_rf):.4f}")


📌 Best Params - Random Forest
{'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}
              precision    recall  f1-score   support

           0       0.89      0.78      0.83      1035
           1       0.55      0.74      0.63       374

    accuracy                           0.77      1409
   macro avg       0.72      0.76      0.73      1409
weighted avg       0.80      0.77      0.78      1409

ROC AUC: 0.8426


In [5]:
xgb_clf = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=(y == 0).sum() / (y == 1).sum(),
    random_state=42
)

param_grid_xgb = {
    "n_estimators": [100, 200],
    "max_depth": [4, 6, 8],
    "learning_rate": [0.01, 0.1, 0.3]
}

grid_xgb = GridSearchCV(xgb_clf, param_grid_xgb, cv=5, scoring='roc_auc', n_jobs=-1)
grid_xgb.fit(X_train, y_train)

print("📌 Best Params - XGBoost")
print(grid_xgb.best_params_)

y_pred_xgb = grid_xgb.predict(X_test)
y_proba_xgb = grid_xgb.predict_proba(X_test)[:, 1]
print(classification_report(y_test, y_pred_xgb))
print(f"ROC AUC: {roc_auc_score(y_test, y_proba_xgb):.4f}")


/Users/nimralatif/Desktop/Portfolio/Customer Churn Prediction System/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:19:43] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/nimralatif/Desktop/Portfolio/Customer Churn Prediction System/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:19:43] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/nimralatif/Desktop/Portfolio/Customer Churn Prediction System/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:19:43] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/nimralatif/Desktop/Portfolio/Customer Churn Prediction

📌 Best Params - XGBoost
{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 200}
              precision    recall  f1-score   support

           0       0.91      0.72      0.80      1035
           1       0.51      0.81      0.63       374

    accuracy                           0.74      1409
   macro avg       0.71      0.76      0.71      1409
weighted avg       0.81      0.74      0.76      1409

ROC AUC: 0.8427


In [6]:
results = {
    "Model": ["Logistic Regression", "Random Forest", "XGBoost"],
    "ROC AUC": [
        roc_auc_score(y_test, y_proba_lr),
        roc_auc_score(y_test, y_proba_rf),
        roc_auc_score(y_test, y_proba_xgb)
    ]
}

pd.DataFrame(results).sort_values(by="ROC AUC", ascending=False)


,Model,ROC AUC
2,XGBoost,0.842724
1,Random Forest,0.842621
0,Logistic Regression,0.840634


In [9]:
results_df = pd.DataFrame(results).sort_values(by="ROC AUC", ascending=False)
print(results_df)

                 Model   ROC AUC
2              XGBoost  0.842724
1        Random Forest  0.842621
0  Logistic Regression  0.840634


In [10]:
# Selecting the best model
best_model_name = results_df.iloc[0]["Model"]
if best_model_name == "Random Forest":
    best_model = grid_rf.best_estimator_
elif best_model_name == "XGBoost":
    best_model = grid_xgb.best_estimator_

In [15]:
# Save the best model
best_model_filename = "../models/best_model.pkl"
joblib.dump(best_model, best_model_filename)
print(f"Best model saved to {best_model_filename}")

Best model saved to ../models/best_model.pkl
